In [1]:
from pathlib import Path
Path("data/input/ma/enrollment/Extracted Data").exists()


True

In [2]:
from pathlib import Path
len(list(Path("data/input/ma/enrollment/Extracted Data").glob("*.csv")))

24

In [3]:
from pathlib import Path

BASE_DIR = Path("data/input/ma/enrollment/Extracted Data")

In [4]:
import pandas as pd

YEAR = 2018
dfs = []

for m in range(1, 13):
    mm = f"{m:02d}"

    contract = pd.read_csv(
        BASE_DIR / f"CPSC_Contract_Info_{YEAR}_{mm}.csv",
        skiprows=1,
        header=None,
        encoding="latin1"
    )

    enroll = pd.read_csv(
        BASE_DIR / f"CPSC_Enrollment_Info_{YEAR}_{mm}.csv",
        skiprows=1,
        header=None,
        na_values=["*"],
        encoding="latin1"
    )

    df = contract.merge(enroll, on=[0, 1], how="left")
    df["month"] = m
    df["year"] = YEAR

    dfs.append(df)

monthly = pd.concat(dfs, ignore_index=True)
print("Monthly shape:", monthly.shape)
monthly.head()
monthly.columns = [
    "contractid", "planid", "org_type", "plan_type", "partd", "snp", "eghp",
    "org_name", "org_marketing_name", "plan_name", "parent_org", "contract_date",
    "ssa", "fips", "state", "county", "enrollment",
    "month", "year"
]
annual = (
    monthly
    .groupby(
        ["contractid", "planid", "plan_type", "state", "county", "year"],
        as_index=False
    )
    .agg(
        avg_enrollment=("enrollment", "mean"),
        snp=("snp", "first"),
        eghp=("eghp", "first")
    )
)

table1 = (
    annual
    .drop_duplicates(["contractid", "planid", "plan_type"])
    .groupby("plan_type")
    .size()
    .reset_index(name="count")
)

table1




Monthly shape: (27710394, 19)


,plan_type,count
0,1876 Cost,101
1,Employer/Union Only Direct Contract PDP,3
2,HMO/HMOPOS,2678
3,Local PPO,966
4,MSA,5
5,Medicare Prescription Drug Plan,1011
6,Medicare-Medicaid Plan HMO/HMOPOS,54
7,National PACE,258
8,PFFS,50
9,Regional PPO,109


In [5]:
annual_q2 = annual[
    (annual["snp"] != "Y") &
    (annual["eghp"] != "Y") &
    (~annual["planid"].between(800, 899))
]

table1_q2 = (
    annual_q2
    .drop_duplicates(["contractid", "planid", "plan_type"])
    .groupby("plan_type")
    .size()
    .reset_index(name="count")
)

table1_q2

,plan_type,count
0,1876 Cost,93
1,HMO/HMOPOS,2133
2,Local PPO,622
3,MSA,3
4,Medicare Prescription Drug Plan,794
5,Medicare-Medicaid Plan HMO/HMOPOS,54
6,National PACE,258
7,PFFS,46
8,Regional PPO,62


In [6]:
service_area = pd.read_csv(
    "data/input/ma/service_area/MA_Cnty_SA_2018_07.csv",
    encoding="latin1"
)
service_area = service_area.rename(columns={
    "Contract ID": "contractid",
    "State": "state",
    "County": "county",
    "Plan Type": "plan_type"
})
annual_q3 = annual_q2.merge(
    service_area,
    on=["contractid", "state", "county", "plan_type"],
    how="inner"
)
table_q3 = (
    annual_q3
    .groupby("plan_type")["avg_enrollment"]
    .mean()
    .reset_index()
    .rename(columns={
        "plan_type": "plan type",
        "avg_enrollment": "average enrollment"
    })
)
table_q3["average enrollment"] = table_q3["average enrollment"].round().astype(int)
table_q3





,plan type,average enrollment
0,1876 Cost,251
1,HMO/HMOPOS,607
2,Local PPO,324
3,MSA,58
4,Medicare-Medicaid Plan HMO/HMOPOS,1019
5,National PACE,150
6,PFFS,94
7,Regional PPO,182


In [7]:
from IPython.display import HTML

HTML("""
<style>
div.input {
    display: none;
}
</style>
""")


In [8]:
!jupyter nbconvert Untitled.ipynb --to html --no-input



[NbConvertApp] Converting notebook Untitled.ipynb to html
[NbConvertApp] Writing 275277 bytes to Untitled.html
